In [16]:
import pandas as pd
import tensorflow as tf

# Automatically reload imported modules that are changed outside this notebook
# More pixels in figures
import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 200


In [17]:
import numpy as np
np_rng = np.random.default_rng(1)

tf.random.set_seed(np_rng.integers(0, tf.int64.max))



import urllib.parse
from IPython.display import display, Markdown

import os

from lidbox.meta import (
    common_voice,
    generate_label2target,
    verify_integrity,
    read_audio_durations,
    random_oversampling_on_split
)

tf.random.set_seed(np_rng.integers(0, tf.int64.max))

train = pd.read_csv("train.tsv", sep="\t")
test = pd.read_csv("test.tsv", sep="\t")
dev = pd.read_csv("dev.tsv", sep="\t")

train["path"] = train["path"].apply(lambda x: x[:-3] + "mp3")
test["path"] = test["path"].apply(lambda x: x[:-3] + "mp3")
dev["path"] = dev["path"].apply(lambda x: x[:-3] + "mp3")

train["split"] = "train"
test["split"] = "test"
dev["split"] = "dev"
#test = test.sample(30000, replace=False)
meta = pd.concat([train, test, dev])


In [18]:
meta.loc[meta["locale"] != "kz", "path"] = "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/" +  meta.loc[meta["locale"] != "kz"]["locale"] + "/clips/" + meta.loc[meta["locale"] != "kz"]["path"]
targets = {"kz": 0, "ru": 1, "en":2, "other":3}
meta["target"] = meta["locale"]
meta.loc[(meta["locale"] != "kz") & (meta["locale"] != "ru") & (meta["locale"]!="en"), "target"] = "other"
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5f590a130a73c.mp3"]
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5ef9bd9ba7029.mp3"]

meta["id"] = meta["Unnamed: 0"].apply(str)
meta["target"] = meta["target"].map(targets)



In [19]:
workdir = "/tf/datasets/augmentedXvector/"


In [20]:
meta["id"] = meta["path"]

In [21]:
import tensorflow_io as tfio

In [22]:
import scipy.signal

from lidbox.features import audio, cmvn
import lidbox.data.steps as ds_steps


TF_AUTOTUNE = tf.data.experimental.AUTOTUNE


def metadata_to_dataset_input(meta):
    return {
        "id": tf.constant(meta.id, tf.string),
        "path": tf.constant(meta.path, tf.string),
        "target": tf.constant(meta.target, tf.int32),
        "split": tf.constant(meta.split, tf.string),
    }

def read_mp3(x):
    s, r = audio.read_mp3(x["path"])
    out_rate = 16000
    s = audio.resample(s, r, out_rate)
    s = audio.peak_normalize(s, dBFS=-3.0)
    s = audio.remove_silence(s, out_rate)
    return dict(x, signal=s, sample_rate=out_rate)


def random_filter(x):
    def scipy_filter(s, N=10):
        b = np_rng.normal(0, 1, N)
        return scipy.signal.lfilter(b, 1.0, s).astype(np.float32), b
    s, _ = tf.numpy_function(
        scipy_filter,
        [x["signal"]],
        [tf.float32, tf.float64],
        name="np_random_filter")
    s = tf.cast(s, tf.float32)
    s = audio.peak_normalize(s, dBFS=-3.0)
    return dict(x, signal=s)


def random_speed_change(ds):
    return ds_steps.random_signal_speed_change(ds, min=0.5, max=1.5, flag=None)


def create_signal_chunks(ds):
    ds = ds_steps.repeat_too_short_signals(ds, 3200)
    ds = ds_steps.create_signal_chunks(ds, 3200, 800)
    return ds


def batch_extract_features(x):
    with tf.device("GPU"):
        signals, rates = x["signal"], x["sample_rate"]
        S = audio.spectrograms(signals, rates[0])
        S = audio.linear_to_mel(S, rates[0])
        S = tf.math.log(S + 1e-6)
        mfccs = tf.signal.mfccs_from_log_mel_spectrograms(S)
        mfccs = mfccs[...,1:21]
        S = cmvn(S, normalize_variance=False)
        mfccs_cmvn = cmvn(mfccs)

        #S = tfio.audio.freq_mask(S, param=10)
        #S = tfio.audio.time_mask(S, param=10)
    return dict(x, logmelspec=S, mfccs=mfccs)


def pipeline_from_meta(data, split):
    if split == "train":
        data = data.sample(frac=1, random_state=np_rng.bit_generator)

    ds = (tf.data.Dataset
            .from_tensor_slices(metadata_to_dataset_input(data))
            .map(read_mp3, num_parallel_calls=TF_AUTOTUNE))

    if split == "train":
        return (ds
            .apply(random_speed_change)
           #.cache(os.path.join(cachedir, "data", split))
            .prefetch(32)
            .map(random_filter, num_parallel_calls=TF_AUTOTUNE)
            .apply(create_signal_chunks)
            .batch(32)
            .map(batch_extract_features, num_parallel_calls=TF_AUTOTUNE)
            .unbatch())
    else:
        return (ds
            .apply(create_signal_chunks)
            .batch(32)
            .map(batch_extract_features, num_parallel_calls=TF_AUTOTUNE)
            .unbatch()
            #.cache(os.path.join(cachedir, "data", split))
            .prefetch(1))


cachedir = os.path.join(workdir, "cache")



In [23]:
val_data = {split: pipeline_from_meta(meta[meta["split"]==split], split)
            for split in meta.split.unique()}
val_data = val_data['dev']

2021-06-27 11:18:18.170 I lidbox.data.steps: Applying random resampling to signals with a random speed ratio chosen uniformly at random from [0.500, 1.500]
2021-06-27 11:18:18.194 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-27 11:18:18.206 I lidbox.data.steps: Dividing every signal in the dataset into new signals by creating signal chunks of length 3200 ms and offset 800 ms. Maximum amount of padding allowed in the last chunk is 0 ms.
2021-06-27 11:18:18.577 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-27 11:18:18.590 I lidbox.data.steps: Dividing every signal in the dataset into new signals by creating signal chunks of length 3200 ms and offset 800 ms. Maximum amount of padding allowed in the last chunk is 0 ms.
2021-06-27 11:18:18.946 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-27 11:18:18.959 I lidbox.data.steps: Dividing every signal in the dataset into new signals by

In [24]:
split2ds = {split: pipeline_from_meta(meta[meta["split"]==split], split)
            for split in meta.split.unique()}


2021-06-27 11:18:19.549 I lidbox.data.steps: Applying random resampling to signals with a random speed ratio chosen uniformly at random from [0.500, 1.500]
2021-06-27 11:18:19.572 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-27 11:18:19.584 I lidbox.data.steps: Dividing every signal in the dataset into new signals by creating signal chunks of length 3200 ms and offset 800 ms. Maximum amount of padding allowed in the last chunk is 0 ms.
2021-06-27 11:18:19.944 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-27 11:18:19.957 I lidbox.data.steps: Dividing every signal in the dataset into new signals by creating signal chunks of length 3200 ms and offset 800 ms. Maximum amount of padding allowed in the last chunk is 0 ms.
2021-06-27 11:18:20.307 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-27 11:18:20.320 I lidbox.data.steps: Dividing every signal in the dataset into new signals by

In [97]:
from tensorflow.keras.layers import (
    Activation,
    BatchNormalization,
    Conv1D,
 
   Dense,
    Dropout,
    Input,
    Layer,
    SpatialDropout1D,
)
from tensorflow.keras.models import Model
import tensorflow as tf

# Assuming spectral features (Batch, Time, Channels), where freq. channels are always last
TIME_AXIS = 1
STDDEV_SQRT_MIN_CLIP = 1e-10


class GlobalMeanStddevPooling1D(Layer):
    """
    Compute arithmetic mean and standard deviation of the inputs along the time steps dimension,
    then output the concatenation of the computed stats.
    """
    def call(self, inputs):
        means = tf.math.reduce_mean(inputs, axis=TIME_AXIS, keepdims=True)
        variances = tf.math.reduce_mean(tf.math.square(inputs - means), axis=TIME_AXIS)
        means = tf.squeeze(means, TIME_AXIS)
        stddevs = tf.math.sqrt(tf.clip_by_value(variances, STDDEV_SQRT_MIN_CLIP, variances.dtype.max))
        return tf.concat((means, stddevs), axis=TIME_AXIS)


def frame_layer(filters, kernel_size, strides, padding="causal", activation="relu", name="frame"):
    return Conv1D(filters, kernel_size, strides, padding=padding, activation=activation, name=name)


def segment_layer(units, activation="relu", name="segment"):
    return Dense(units, activation=activation, name=name)


from tensorflow.keras.layers import (
    Activation,
    BatchNormalization,
    Conv1D,
    Conv2D,
    Dropout,
    Dense,
    GaussianNoise,
    Input,
    Layer,
    LSTM,
    Multiply,
    Reshape,
)

from tensorflow.keras.models import Model
import tensorflow as tf


def frequency_attention(H, d_a=64, d_f=16):
    assert not H.shape[2] % d_f, "amount of frequency channels ({}) must be evenly divisible by the amount of frequency attention bins (d_f={})".format(H.shape[2], d_f)
    # Note, we assume that H.shape = (batch_size, T, d_h), but the paper assumes the timesteps come last
    x = Dense(d_a, activation="relu", use_bias=False, name="Wf_1")(H)
    F_A = Dense(d_f, activation="softmax", use_bias=False, name="Wf_2")(x)
    # Apply frequency attention on d_f bins
    F_A = Reshape((F_A.shape[1] or -1, F_A.shape[2], 1), name="expand_bin_weight_dim")(F_A)
    H_bins = Reshape((H.shape[1] or -1, d_f, H.shape[2] // d_f), name="partition_freq_bins")(H)
    H_bins = Multiply(name="freq_attention")([F_A, H_bins])
    # Merge weighted frequency bins
    H_weighted = Reshape((H.shape[1] or -1, H.shape[2]), name="merge_weighted_bins")(H_bins)
    return H_weighted



from tensorflow.keras.layers import (
    Activation,
    Dense,
    Input,
)

from tensorflow.keras.models import Model
import tensorflow as tf



def create(input_shape, num_outputs, output_activation="log_softmax", freq_attention_bins=60):
    inputs = Input(shape=input_shape, name="input")

    x = frame_layer(512, 5, 1, name="frame1")(inputs)
    x = frame_layer(512, 3, 2, name="frame2")(x)
    x = frame_layer(512, 3, 3, name="frame3")(x)
    x = frame_layer(512, 1, 1, name="frame4")(x)
    x = frame_layer(1500, 1, 1, name="frame5")(x)

    x = frequency_attention(x, d_f=freq_attention_bins)

    x = GlobalMeanStddevPooling1D(name="stats_pooling")(x)

    x = segment_layer(512, name="segment1")(x)
    x = segment_layer(512, name="segment2")(x)

    
    return Model(inputs=inputs, outputs=x, name="x-vector-frequency-attention")

In [98]:
def create_model(num_freq_bins=40, num_labels=len(np.unique(meta.target))):
    m = create(
        input_shape=[None, num_freq_bins],
        num_outputs=num_labels)
    """
    m.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
            metrics=tf.keras.metrics.sparse_categorical_accuracy)
    """
    return m

with tf.device("GPU"):
    model = create_model()
    model.summary()
   

Model: "x-vector-frequency-attention"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, None, 40)]   0                                            
__________________________________________________________________________________________________
frame1 (Conv1D)                 (None, None, 512)    102912      input[0][0]                      
__________________________________________________________________________________________________
frame2 (Conv1D)                 (None, None, 512)    786944      frame1[0][0]                     
__________________________________________________________________________________________________
frame3 (Conv1D)                 (None, None, 512)    786944      frame2[0][0]                     
_______________________________________________________________________

In [110]:
model.trainable = False
x = model.layers[-1].output
x = Dense(128, activation = "relu")(x)
predictions = Dense(4, activation = "softmax")(x)
clf1 = Model(inputs = model.input, outputs = predictions, name="clf1")

In [111]:
clf1.summary()

Model: "clf1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, None, 40)]   0                                            
__________________________________________________________________________________________________
frame1 (Conv1D)                 (None, None, 512)    102912      input[0][0]                      
__________________________________________________________________________________________________
frame2 (Conv1D)                 (None, None, 512)    786944      frame1[0][0]                     
__________________________________________________________________________________________________
frame3 (Conv1D)                 (None, None, 512)    786944      frame2[0][0]                     
_______________________________________________________________________________________________

In [112]:
for l in clf1.layers:
    if l.trainable:
        print(l.name)

dense_24
dense_25


In [113]:
clf1.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            metrics=tf.keras.metrics.sparse_categorical_accuracy)

In [114]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
            metrics=tf.keras.metrics.sparse_categorical_accuracy)



In [104]:
for l in model.layers:
    if l.trainable:
        print(l)
model.summary()

Model: "x-vector-frequency-attention"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, None, 40)]   0                                            
__________________________________________________________________________________________________
frame1 (Conv1D)                 (None, None, 512)    102912      input[0][0]                      
__________________________________________________________________________________________________
frame2 (Conv1D)                 (None, None, 512)    786944      frame1[0][0]                     
__________________________________________________________________________________________________
frame3 (Conv1D)                 (None, None, 512)    786944      frame2[0][0]                     
_______________________________________________________________________

In [115]:
model.trainable = False
x = model.layers[-1].output
x = Dense(128, activation = "relu")(x)
predictions = Dense(4, activation = "softmax")(x)
clf2 = Model(inputs = model.input, outputs = predictions, name="clf2")

In [116]:
clf2.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            metrics=tf.keras.metrics.sparse_categorical_accuracy)

In [117]:
clf2.trainable = True
model.trainable = False
for l in clf2.layers:
    if l.trainable:
        print(l.name)

dense_26
dense_27


## Testing on common voice

In [108]:
meta = meta.set_index('id')

In [109]:
clf1 = tf.keras.models.load_model('clf1_2/model')

clf2 = tf.keras.models.load_model('clf2_2/model')

In [61]:

def predictions_to_dataframe(ids, predictions):
    return (pd.DataFrame.from_dict({"id": ids, "prediction": predictions})
            #.set_index("id", drop=True, verify_integrity=True)
            #.sort_index()
           )

def predict_with_model(model, ds, predict_fn=None):
    """
    Map callable model over all batches in ds, predicting values for each element at key 'input'.
    """
    if predict_fn is None:
        def predict_fn(x):
            with tf.device("GPU"):
                return x["id"], model(x["input"], training=False)

    ids = []
    predictions = []
    for id, pred in ds.map(predict_fn, num_parallel_calls=TF_AUTOTUNE).unbatch().as_numpy_iterator():
        ids.append(id.decode("utf-8"))
        predictions.append(pred)

    return predictions_to_dataframe(ids, predictions)

### Testing CLF1

In [62]:
chunk2pred = predict_with_model(
    model=clf1,
    ds=split2ds["test"].map(lambda x: dict(x, input=x["logmelspec"])).batch(32),
    #predict_fn=predict_with_ap_loss
    )



In [63]:
chunk2pred = chunk2pred.set_index("id")

In [64]:
from lidbox.util import merge_chunk_predictions


utt2pred = merge_chunk_predictions(chunk2pred)
utt2pred

,prediction
id,
/tf/datasets/vox/en_test/-BwrRlUdfEs__U__S0---0003.940-0020.570.mp3,"[0.74763554, 0.015166172, 0.11314385, 0.12405445]"
/tf/datasets/vox/en_test/-BwrRlUdfEs__U__S0---0743.730-0757.100.mp3,"[0.6527672, 0.015340386, 0.17765337, 0.15423909]"
/tf/datasets/vox/en_test/-BwrRlUdfEs__U__S100---0650.920-0661.560.mp3,"[0.979149, 0.00094241154, 0.011158215, 0.00875..."
/tf/datasets/vox/en_test/-BwrRlUdfEs__U__S100---0692.790-0704.510.mp3,"[0.93517745, 0.010677641, 0.023704082, 0.03044..."
/tf/datasets/vox/en_test/-BwrRlUdfEs__U__S100---0705.010-0711.610.mp3,"[0.8223988, 0.036917165, 0.056642402, 0.08404159]"
...,...
/tf/datasets/vox/ru_test/ztSbqN-mPtM__U__S20---0219.180-0230.690.mp3,"[0.9637271, 0.005408268, 0.011009475, 0.019855..."
/tf/datasets/vox/ru_test/ztSbqN-mPtM__U__S20---0230.690-0247.370.mp3,"[0.9565367, 0.017204842, 0.008147814, 0.018110..."
/tf/datasets/vox/ru_test/ztSbqN-mPtM__U__S20---0247.370-0257.750.mp3,"[0.9306425, 0.009629078, 0.031984303, 0.027744..."


### Results of CLF1

In [65]:


from sklearn.metrics import classification_report

test_meta = meta[meta["split"]=="test"].join(utt2pred, how="outer")
assert not test_meta.isna().any(axis=None), "failed to join predictions"

true_sparse = test_meta.target.to_numpy(np.int32)
pred_dense = np.stack(test_meta.prediction.apply(np.argmax))

report = classification_report(true_sparse, pred_dense, target_names=list(targets.keys()), labels=range(4))
print(report)



              precision    recall  f1-score   support

          kz       0.39      0.98      0.56     13946
          ru       0.75      0.01      0.01     12107
          en       0.64      0.01      0.02     10000
       other       0.00      0.00      0.00         0

    accuracy                           0.38     36053
   macro avg       0.44      0.25      0.15     36053
weighted avg       0.58      0.38      0.22     36053



/root/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Testing clf2

In [31]:
chunk2pred = predict_with_model(
    model=clf2,
    ds=split2ds["test"].map(lambda x: dict(x, input=x["logmelspec"])).batch(32),
    #predict_fn=predict_with_ap_loss
    )



In [32]:
chunk2pred = chunk2pred.set_index("id")

In [33]:
from lidbox.util import merge_chunk_predictions


utt2pred = merge_chunk_predictions(chunk2pred)
utt2pred

,prediction
id,
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/cs/clips/common_voice_cs_20424383.mp3,"[8.308059e-09, 0.0003079729, 6.093462e-05, 0.9..."
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/cs/clips/common_voice_cs_20424555.mp3,"[2.471661e-07, 0.005254525, 0.00096489495, 0.9..."
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/cs/clips/common_voice_cs_20424567.mp3,"[1.87723e-07, 0.80234873, 5.8355516e-05, 0.197..."
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/cs/clips/common_voice_cs_20424609.mp3,"[1.9055422e-05, 0.7478529, 1.27799485e-05, 0.2..."
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/cs/clips/common_voice_cs_20424636.mp3,"[2.5810184e-06, 0.056320507, 0.001680822, 0.94..."
...,...
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/zh-CN/clips/common_voice_zh-CN_22242585.mp3,"[3.313049e-08, 8.7356224e-05, 0.0008840677, 0...."
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/zh-CN/clips/common_voice_zh-CN_22242819.mp3,"[0.00059783406, 0.08054378, 0.7682407, 0.1506177]"
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/zh-CN/clips/common_voice_zh-CN_22243431.mp3,"[1.5969928e-05, 0.0027630564, 0.12403288, 0.87..."


### Results of clf2

In [34]:


from sklearn.metrics import classification_report

test_meta = meta[meta["split"]=="test"].join(utt2pred, how="outer")
assert not test_meta.isna().any(axis=None), "failed to join predictions"

true_sparse = test_meta.target.to_numpy(np.int32)
pred_dense = np.stack(test_meta.prediction.apply(np.argmax))

report = classification_report(true_sparse, pred_dense, target_names=list(targets.keys()), labels=range(4))
print(report)



              precision    recall  f1-score   support

          kz       1.00      1.00      1.00     17341
          ru       0.88      0.75      0.81     10379
          en       0.87      0.87      0.87     12964
       other       0.77      0.84      0.80     15084

    accuracy                           0.88     55768
   macro avg       0.88      0.86      0.87     55768
weighted avg       0.88      0.88      0.88     55768



Since the results vary, it is reasonable to conduct the experiment further

## Part 2. Training the classifiers to increase the discrepancy on the VOX data

In [46]:
import numpy as np
np_rng = np.random.default_rng(1)

tf.random.set_seed(np_rng.integers(0, tf.int64.max))



import urllib.parse
from IPython.display import display, Markdown

import os

from lidbox.meta import (
    common_voice,
    generate_label2target,
    verify_integrity,
    read_audio_durations,
    random_oversampling_on_split
)

tf.random.set_seed(np_rng.integers(0, tf.int64.max))

train = pd.read_csv("train.tsv", sep="\t")
test = pd.read_csv("new_test.tsv", sep="\t")
dev = pd.read_csv("new_dev.tsv", sep="\t")


train["path"] = train["path"].apply(lambda x: x[:-3] + "mp3")
test["path"] = test["path"].apply(lambda x: x[:-3] + "mp3")
dev["path"] = dev["path"].apply(lambda x: x[:-3] + "mp3")

train["split"] = "train"
test["split"] = "test"
dev["split"] = "dev"
meta = pd.concat([train, test, dev])


In [47]:
meta.loc[((meta["locale"] != "kz") & ~(((meta["split"] == "dev") | (meta["split"] == "test")) & ((meta["locale"] == "ru") | (meta["locale"] == "kz") | (meta["locale"] == "en")))), "path"] = "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/" + meta.loc[((meta["locale"] != "kz") & ~(((meta["split"] == "dev") | (meta["split"] == "test")) & ((meta["locale"] == "ru") | (meta["locale"] == "kz") | (meta["locale"] == "en"))))]["locale"]  + "/clips/" + meta.loc[((meta["locale"] != "kz") & ~(((meta["split"] == "dev") | (meta["split"] == "test")) & ((meta["locale"] == "ru") | (meta["locale"] == "kz") | (meta["locale"] == "en"))))]["path"]
targets = {"kz": 0, "ru": 1, "en":2, "other":3}
meta["target"] = meta["locale"]
meta.loc[(meta["locale"] != "kz") & (meta["locale"] != "ru") & (meta["locale"]!="en"), "target"] = "other"
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5f590a130a73c.mp3"]
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5ef9bd9ba7029.mp3"]

meta["id"] = str(meta["Unnamed: 0"])
meta["target"] = meta["target"].map(targets)




In [48]:
meta.loc[(meta["split"] == "test") & (meta["locale"] == "ru"), "path"] = meta.loc[(meta["split"] == "test") & (meta["locale"] == "ru")]["path"].apply(lambda x: f"/tf/datasets/vox/ru_test/{x}")
meta.loc[(meta["split"] == "test") & (meta["locale"] == "ru"), "path"]
meta.loc[(meta["split"] == "test") & (meta["locale"] == "kz"), "path"] = meta.loc[(meta["split"] == "test") & (meta["locale"] == "kz")]["path"].apply(lambda x: f"/tf/datasets/vox/kz_test/{x}")
meta.loc[(meta["split"] == "test") & (meta["locale"] == "kz"), "path"] 
meta.loc[(meta["split"] == "test") & (meta["locale"] == "en"), "path"] = meta.loc[(meta["split"] == "test") & (meta["locale"] == "en")]["path"].apply(lambda x: f"/tf/datasets/vox/en_test/{x}")
meta.loc[(meta["split"] == "test") & (meta["locale"] == "en"), "path"] 

0       /tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...
1       /tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123-...
2       /tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---...
3       /tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---...
4       /tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---...
                              ...                        
9995    /tf/datasets/vox/en_test/KLiy94kfZI4__U__S133-...
9996    /tf/datasets/vox/en_test/YTlliEr5LOA__U__S113-...
9997    /tf/datasets/vox/en_test/bSs0gNq6Kkc__U__S0---...
9998    /tf/datasets/vox/en_test/Da7c-BY6MDA__U__S2---...
9999    /tf/datasets/vox/en_test/VWvPndMo1F8__U__S24--...
Name: path, Length: 10000, dtype: object

In [49]:
meta.loc[(meta["split"] == "dev") & (meta["locale"] == "ru"), "path"] = meta.loc[(meta["split"] == "dev") & (meta["locale"] == "ru")]["path"].apply(lambda x: f"/tf/datasets/vox/ru_dev/{x}")
meta.loc[(meta["split"] == "dev") & (meta["locale"] == "ru"), "path"]
meta.loc[(meta["split"] == "dev") & (meta["locale"] == "kz"), "path"] = meta.loc[(meta["split"] == "dev") & (meta["locale"] == "kz")]["path"].apply(lambda x: f"/tf/datasets/vox/kz_dev/{x}")
meta.loc[(meta["split"] == "dev") & (meta["locale"] == "kz"), "path"] 
meta.loc[(meta["split"] == "dev") & (meta["locale"] == "en"), "path"] = meta.loc[(meta["split"] == "dev") & (meta["locale"] == "en")]["path"].apply(lambda x: f"/tf/datasets/vox/en_dev/{x}")
meta.loc[(meta["split"] == "dev") & (meta["locale"] == "en"), "path"] 

10106    /tf/datasets/vox/en_dev/Jjwf054bhTU__U__S0---0...
10107    /tf/datasets/vox/en_dev/6TEi0nJP5BE__U__S18---...
10108    /tf/datasets/vox/en_dev/Zq4wKUsd0ZM__U__S10---...
10109    /tf/datasets/vox/en_dev/8HzZXIchBhQ__U__S27---...
10110    /tf/datasets/vox/en_dev/-J43Gvlztc8__U__S37---...
                               ...                        
15961    /tf/datasets/vox/en_dev/SHU3RJ03QlM__U__S1---0...
15962    /tf/datasets/vox/en_dev/4wFO1Qqi3nQ__U__S0---0...
15963    /tf/datasets/vox/en_dev/4zWSmjij-hs__U__S54---...
15964    /tf/datasets/vox/en_dev/bi8wo64I57U__U__S0---0...
15965    /tf/datasets/vox/en_dev/pqAQ4JkeOZI__U__S2---0...
Name: path, Length: 5860, dtype: object

In [50]:
meta.loc[meta["split"]=="test", "Unnamed: 0"] = meta.loc[meta["split"]=="test"]["path"]
meta.loc[meta["split"]=="dev", "Unnamed: 0"] = meta.loc[meta["split"]=="dev"]["path"]

In [51]:
meta["id"] = meta["Unnamed: 0"].apply(str)

In [52]:
meta.loc[meta["split"] == "test", "id"] = meta.loc[meta["split"] == "test"]["path"]

In [53]:
meta = meta.set_index("Unnamed: 0")
meta.loc[meta["split"]=="dev"]

,path,locale,split,target,id
Unnamed: 0,,,,,
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/ta/clips/common_voice_ta_19093662.mp3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,ta,dev,3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/es/clips/common_voice_es_20252168.mp3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,es,dev,3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/it/clips/common_voice_it_23740290.mp3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,it,dev,3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/pl/clips/common_voice_pl_22062475.mp3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,pl,dev,3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/fr/clips/common_voice_fr_23921769.mp3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,fr,dev,3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...
...,...,...,...,...,...
/tf/datasets/vox/ru_dev/BH8c4SbgXss__U__S251---1604.030-1609.420.mp3,/tf/datasets/vox/ru_dev/BH8c4SbgXss__U__S251--...,ru,dev,1,/tf/datasets/vox/ru_dev/BH8c4SbgXss__U__S251--...
/tf/datasets/vox/ru_dev/--RxvUW3u7M__U__S0---0552.770-0565.180.mp3,/tf/datasets/vox/ru_dev/--RxvUW3u7M__U__S0---0...,ru,dev,1,/tf/datasets/vox/ru_dev/--RxvUW3u7M__U__S0---0...
/tf/datasets/vox/ru_dev/kZ8LKE26cl0__U__S1---0100.470-0114.760.mp3,/tf/datasets/vox/ru_dev/kZ8LKE26cl0__U__S1---0...,ru,dev,1,/tf/datasets/vox/ru_dev/kZ8LKE26cl0__U__S1---0...


In [54]:
meta.loc[meta["split"] == "test"] = meta.loc[(meta["split"] == "test") & (meta["target"] != 3)] 


In [55]:
split2ds = {split: pipeline_from_meta(meta[meta["split"]==split].sample(frac=1.0), split)
            for split in meta.split.unique()}

2021-06-27 13:53:12.985 I lidbox.data.steps: Applying random resampling to signals with a random speed ratio chosen uniformly at random from [0.500, 1.500]
2021-06-27 13:53:13.109 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-27 13:53:13.122 I lidbox.data.steps: Dividing every signal in the dataset into new signals by creating signal chunks of length 3200 ms and offset 800 ms. Maximum amount of padding allowed in the last chunk is 0 ms.
2021-06-27 13:53:13.608 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-27 13:53:13.620 I lidbox.data.steps: Dividing every signal in the dataset into new signals by creating signal chunks of length 3200 ms and offset 800 ms. Maximum amount of padding allowed in the last chunk is 0 ms.
2021-06-27 13:53:13.961 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-27 13:53:13.974 I lidbox.data.steps: Dividing every signal in the dataset into new signals by

### But first of all test on the VOX data

In [118]:
from tqdm import tqdm
EPOCHS = 100 
dev_iterator = iter(split2ds["dev"].batch(32).repeat(1000))
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4)
optimizer2 = tf.keras.optimizers.Adam(learning_rate=1e-4)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
best_acc = 0
counter = 0
MAX_PATIENCE = 5
c = 0

def discrepancy(out1, out2):
    return tf.reduce_mean(tf.abs(out1 - out2))

for epoch in range(EPOCHS):
    print(f"It is epoch number: {epoch}")
    avg_loss = 0
    for i in tqdm(split2ds["train"].batch(32)):
        c += 1
        clf1.trainable = True
        clf2.trainable = True
        model.trainable = True
        target_data = next(dev_iterator)
        s_specs = i['logmelspec']
        t_specs = target_data['logmelspec']
        # Train for classification
        with tf.GradientTape() as tape:
            pred1 = clf1(s_specs)
            pred2 = clf2(s_specs)
            loss_classification_1 = loss_fn(i['target'], pred1)
            loss_classification_2 = loss_fn(i['target'], pred2)
            total_classification_loss = loss_classification_1 + loss_classification_2
            avg_loss += total_classification_loss
            
        train_acc_metric.update_state(i['target'], pred1)
        grads = tape.gradient(total_classification_loss, [clf1.trainable_weights, clf2.trainable_weights, model.trainable_weights])
        optimizer.apply_gradients(zip(grads[0], clf1.trainable_weights))
        optimizer.apply_gradients(zip(grads[1], clf2.trainable_weights))
        optimizer.apply_gradients(zip(grads[2], model.trainable_weights))
        # Train for discrepancy increase
        model.trainable = False
        with tf.GradientTape() as tape2:
            pred1 = clf1(s_specs)
            pred2 = clf2(s_specs)
            loss_classification_1 = loss_fn(i['target'], pred1)
            loss_classification_2 = loss_fn(i['target'], pred2)
            total_classification_loss = loss_classification_1 + loss_classification_2
            pred1 = clf1(t_specs)
            pred2 = clf2(t_specs)
            discrepancy_loss = discrepancy(pred1, pred2)
            dl1 = discrepancy_loss
            loss = total_classification_loss - discrepancy_loss

        grads = tape2.gradient(loss, [clf1.trainable_weights, clf2.trainable_weights])
        optimizer.apply_gradients(zip(grads[0], clf1.trainable_weights))
        optimizer.apply_gradients(zip(grads[1], clf2.trainable_weights))
        # Train for discrepancy decrease
        clf1.trainable = False
        clf2.trainable = False
        model.trainable = True
        dloss = 0
        for k in range(3):
            with tf.GradientTape() as tape3:
                pred1 = clf1(t_specs)
                pred2 = clf2(t_specs)
                discrepancy_loss = discrepancy(pred1, pred2)
                dloss = discrepancy_loss
            grads = tape3.gradient(discrepancy_loss, model.trainable_weights)            
            optimizer2.apply_gradients(zip(grads, model.trainable_weights))
        if c % 1000 == 0:
            tqdm.write(f"loss: {total_classification_loss}, discrepancy before: {dl1}, discrepancy loss after: {dloss}")
            c = 0
    for batch in tqdm(val_data.batch(32)):
        val_preds = clf1(batch['logmelspec'])
        true_vals = batch['target']  
        val_acc_metric.update_state(true_vals, val_preds)
    new_acc = 0
    val_acc = val_acc_metric.result()
    new_acc += val_acc
    val_acc_metric.reset_states()
    train_acc = train_acc_metric.result()
    train_acc_metric.reset_states()
    print(f"Val acc: {val_acc}, Train acc: {train_acc}. Train loss (clf1): {avg_loss/9783}")
    for batch in tqdm(val_data.batch(32)):
        val_preds = clf2(batch['logmelspec'])
        true_vals = batch['target']  
        val_acc_metric.update_state(true_vals, val_preds)

    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    train_acc = train_acc_metric.result()
    train_acc_metric.reset_states()
    print(f"Val acc (clf2): {val_acc}.")
    new_acc += val_acc
    if new_acc / 2 > best_acc:
        print(f"Here is an improvement from {best_acc} to {new_acc / 2}\nSaving results")
        best_acc = new_acc / 2
        counter = 0
        clf1.save("clf1_3/model")
        clf2.save("clf2_3/model")
        model.save("extractor/model")
    else:
        print("No improvements")
        counter += 1
    if counter >= MAX_PATIENCE:
        break

0it [00:00, ?it/s]

It is epoch number: 0


1000it [04:31,  3.72it/s]

loss: 1.6472115516662598, discrepancy before: 0.056961655616760254, discrepancy loss after: 0.057231366634368896


2000it [08:58,  3.80it/s]

...

loss: 1.7237623929977417, discrepancy before: 0.003573259338736534, discrepancy loss after: 0.0035681030713021755


4446it [19:46,  3.81it/s]

loss: 0.8444103002548218, discrepancy before: 0.002823175862431526, discrepancy loss after: 0.0025392004754394293


4564it [20:18,  3.75it/s]


KeyboardInterrupt: 

In [119]:
target_data = next(dev_iterator)
t_specs = target_data['logmelspec']
print(clf2(t_specs), target_data['target'])

tf.Tensor(
[[1.00000000e+00 1.19009230e-14 1.32020670e-11 1.01135653e-13]
 [1.00000000e+00 2.16983809e-11 4.74420592e-09 1.34875458e-10]
 [9.99999881e-01 1.05067799e-09 9.92498244e-08 5.55637492e-09]
 [1.00000000e+00 6.37759429e-11 1.10437242e-08 3.79018678e-10]
 [9.99997497e-01 5.62239713e-08 2.24551172e-06 2.51913491e-07]
 [1.00000000e+00 2.81078546e-12 9.56210111e-10 1.90240705e-11]
 [9.94534254e-01 5.77919185e-04 3.12577211e-03 1.76195463e-03]
 [9.99553859e-01 3.03773759e-05 3.11050972e-04 1.04725972e-04]
 [9.99991536e-01 2.46231849e-07 7.14492262e-06 1.03742855e-06]
 [9.99998927e-01 2.08856150e-08 1.03339642e-06 9.75192265e-08]
 [9.99999404e-01 1.03782938e-08 5.97366068e-07 4.98906942e-08]
 [9.99999881e-01 7.76933629e-10 7.83425449e-08 4.16069090e-09]
 [9.99990582e-01 2.83229298e-07 7.97335997e-06 1.18636899e-06]
 [1.00000000e+00 4.66268274e-11 8.64002647e-09 2.80740209e-10]
 [9.99965429e-01 1.38509154e-06 2.76613573e-05 5.43059423e-06]
 [9.99775350e-01 1.33777503e-05 1.63577890e-

In [120]:
clf2.trainable_weights

[<tf.Variable 'dense_26/kernel:0' shape=(512, 128) dtype=float32, numpy=
 array([[-0.04428463, -0.06575986, -0.03093002, ..., -0.06438525,
         -0.06454992, -0.03146683],
        [-0.06528097, -0.06506743, -0.15744567, ..., -0.09292911,
         -0.01714528, -0.01219531],
        [ 0.0603695 , -0.04678904, -0.00422427, ..., -0.08944634,
          0.06902291, -0.05208739],
        ...,
        [-0.14467703, -0.0124551 , -0.11538287, ...,  0.01766187,
          0.11588152,  0.07392925],
        [-0.02426761,  0.04343619,  0.1003718 , ..., -0.0510636 ,
         -0.12005099,  0.0778337 ],
        [ 0.02266992,  0.03148512,  0.04171593, ..., -0.03963193,
         -0.00856541, -0.01874579]], dtype=float32)>,
 <tf.Variable 'dense_26/bias:0' shape=(128,) dtype=float32, numpy=
 array([ 0.35369572, -0.07690878, -0.16411977, -0.10389522, -0.03021821,
        -0.02824787, -0.18758969, -0.14436996,  0.0174201 , -0.10246152,
        -0.17557439,  0.45739302, -0.14029697, -0.16773385, -0.16950361

In [128]:
chunk2pred = predict_with_model(
    model=clf2,
    ds=split2ds["test"].map(lambda x: dict(x, input=x["logmelspec"])).batch(32),
    #predict_fn=predict_with_ap_loss
    )



In [129]:
chunk2pred = chunk2pred.set_index("id")

In [130]:
meta

,path,locale,split,target
id,,,,
1486,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,ru,train,1.0
56701,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,kz,train,0.0
3364,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,ru,train,1.0
110475,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,rw,train,3.0
45384,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,en,train,2.0
...,...,...,...,...
/tf/datasets/vox/ru_dev/BH8c4SbgXss__U__S251---1604.030-1609.420.mp3,/tf/datasets/vox/ru_dev/BH8c4SbgXss__U__S251--...,ru,dev,1.0
/tf/datasets/vox/ru_dev/--RxvUW3u7M__U__S0---0552.770-0565.180.mp3,/tf/datasets/vox/ru_dev/--RxvUW3u7M__U__S0---0...,ru,dev,1.0
/tf/datasets/vox/ru_dev/kZ8LKE26cl0__U__S1---0100.470-0114.760.mp3,/tf/datasets/vox/ru_dev/kZ8LKE26cl0__U__S1---0...,ru,dev,1.0


In [131]:
from lidbox.util import merge_chunk_predictions


utt2pred = merge_chunk_predictions(chunk2pred)
utt2pred

,prediction
id,
/tf/datasets/vox/en_test/-BwrRlUdfEs__U__S0---0003.940-0020.570.mp3,"[0.82074064, 0.036507558, 0.05600509, 0.086746..."
/tf/datasets/vox/en_test/-BwrRlUdfEs__U__S0---0743.730-0757.100.mp3,"[0.7609731, 0.050528564, 0.07821112, 0.11028719]"
/tf/datasets/vox/en_test/-BwrRlUdfEs__U__S100---0650.920-0661.560.mp3,"[0.99982405, 1.17218315e-05, 0.00012380008, 4...."
/tf/datasets/vox/en_test/-BwrRlUdfEs__U__S100---0692.790-0704.510.mp3,"[0.99906653, 9.269332e-05, 0.0005545609, 0.000..."
/tf/datasets/vox/en_test/-BwrRlUdfEs__U__S100---0705.010-0711.610.mp3,"[0.9915779, 0.0011870286, 0.0039456706, 0.0032..."
...,...
/tf/datasets/vox/ru_test/ztSbqN-mPtM__U__S20---0219.180-0230.690.mp3,"[0.9704744, 0.0048984527, 0.01188823, 0.012738..."
/tf/datasets/vox/ru_test/ztSbqN-mPtM__U__S20---0230.690-0247.370.mp3,"[0.9708305, 0.005451752, 0.0102613745, 0.01345..."
/tf/datasets/vox/ru_test/ztSbqN-mPtM__U__S20---0247.370-0257.750.mp3,"[0.9644639, 0.006060426, 0.01391795, 0.015557733]"


In [132]:


from sklearn.metrics import classification_report

test_meta = meta[meta["split"]=="test"].join(utt2pred, how="outer")
assert not test_meta.isna().any(axis=None), "failed to join predictions"

true_sparse = test_meta.target.to_numpy(np.int32)
pred_dense = np.stack(test_meta.prediction.apply(np.argmax))

report = classification_report(true_sparse, pred_dense, target_names=list(targets.keys()), labels=range(4))
print(report)



              precision    recall  f1-score   support

          kz       0.39      0.99      0.55     13946
          ru       0.00      0.00      0.00     12107
          en       0.47      0.00      0.00     10000
       other       0.00      0.00      0.00         0

    accuracy                           0.38     36053
   macro avg       0.21      0.25      0.14     36053
weighted avg       0.28      0.38      0.22     36053



/root/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
import pandas as pd
import tensorflow as tf

# Automatically reload imported modules that are changed outside this notebook
# More pixels in figures
import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 200

import numpy as np
np_rng = np.random.default_rng(1)

tf.random.set_seed(np_rng.integers(0, tf.int64.max))



import urllib.parse
from IPython.display import display, Markdown

import os

from lidbox.meta import (
    common_voice,
    generate_label2target,
    verify_integrity,
    read_audio_durations,
    random_oversampling_on_split
)

tf.random.set_seed(np_rng.integers(0, tf.int64.max))

train = pd.read_csv("train.tsv", sep="\t")
test = pd.read_csv("test.tsv", sep="\t")
dev = pd.read_csv("dev.tsv", sep="\t")

train["path"] = train["path"].apply(lambda x: x[:-3] + "mp3")
test["path"] = test["path"].apply(lambda x: x[:-3] + "mp3")
dev["path"] = dev["path"].apply(lambda x: x[:-3] + "mp3")

train["split"] = "train"
test["split"] = "test"
dev["split"] = "dev"
#test = test.sample(30000, replace=False)
meta = pd.concat([train, test, dev])

meta.loc[meta["locale"] != "kz", "path"] = "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/" +  meta.loc[meta["locale"] != "kz"]["locale"] + "/clips/" + meta.loc[meta["locale"] != "kz"]["path"]
targets = {"kz": 0, "ru": 1, "en":2, "other":3}
meta["target"] = meta["locale"]
meta.loc[(meta["locale"] != "kz") & (meta["locale"] != "ru") & (meta["locale"]!="en"), "target"] = "other"
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5f590a130a73c.mp3"]
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5ef9bd9ba7029.mp3"]

meta["id"] = meta["Unnamed: 0"].apply(str)
meta["target"] = meta["target"].map(targets)


meta["id"] = meta["path"]

import tensorflow_io as tfio

import scipy.signal

from lidbox.features import audio, cmvn
import lidbox.data.steps as ds_steps


TF_AUTOTUNE = tf.data.experimental.AUTOTUNE


def metadata_to_dataset_input(meta):
    return {
        "id": tf.constant(meta.id, tf.string),
        "path": tf.constant(meta.path, tf.string),
        "target": tf.constant(meta.target, tf.int32),
        "split": tf.constant(meta.split, tf.string),
    }

def read_mp3(x):
    s, r = audio.read_mp3(x["path"])
    out_rate = 16000
    s = audio.resample(s, r, out_rate)
    s = audio.peak_normalize(s, dBFS=-3.0)
    s = audio.remove_silence(s, out_rate)
    return dict(x, signal=s, sample_rate=out_rate)


def random_filter(x):
    def scipy_filter(s, N=10):
        b = np_rng.normal(0, 1, N)
        return scipy.signal.lfilter(b, 1.0, s).astype(np.float32), b
    s, _ = tf.numpy_function(
        scipy_filter,
        [x["signal"]],
        [tf.float32, tf.float64],
        name="np_random_filter")
    s = tf.cast(s, tf.float32)
    s = audio.peak_normalize(s, dBFS=-3.0)
    return dict(x, signal=s)


def random_speed_change(ds):
    return ds_steps.random_signal_speed_change(ds, min=0.5, max=1.5, flag=None)


def create_signal_chunks(ds):
    ds = ds_steps.repeat_too_short_signals(ds, 3200)
    ds = ds_steps.create_signal_chunks(ds, 3200, 800)
    return ds


def batch_extract_features(x):
    with tf.device("GPU"):
        signals, rates = x["signal"], x["sample_rate"]
        S = audio.spectrograms(signals, rates[0])
        S = audio.linear_to_mel(S, rates[0])
        S = tf.math.log(S + 1e-6)
        mfccs = tf.signal.mfccs_from_log_mel_spectrograms(S)
        mfccs = mfccs[...,1:21]
        S = cmvn(S, normalize_variance=False)
        mfccs_cmvn = cmvn(mfccs)

        #S = tfio.audio.freq_mask(S, param=10)
        #S = tfio.audio.time_mask(S, param=10)
    return dict(x, logmelspec=S, mfccs=mfccs)


def pipeline_from_meta(data, split):
    if split == "train":
        data = data.sample(frac=1, random_state=np_rng.bit_generator)

    ds = (tf.data.Dataset
            .from_tensor_slices(metadata_to_dataset_input(data))
            .map(read_mp3, num_parallel_calls=TF_AUTOTUNE))

    if split == "train":
        return (ds
            .apply(random_speed_change)
           #.cache(os.path.join(cachedir, "data", split))
            .prefetch(32)
            .map(random_filter, num_parallel_calls=TF_AUTOTUNE)
            .apply(create_signal_chunks)
            .batch(32)
            .map(batch_extract_features, num_parallel_calls=TF_AUTOTUNE)
            .unbatch())
    else:
        return (ds
            .apply(create_signal_chunks)
            .batch(32)
            .map(batch_extract_features, num_parallel_calls=TF_AUTOTUNE)
            .unbatch()
            #.cache(os.path.join(cachedir, "data", split))
            .prefetch(1))


val_data = {split: pipeline_from_meta(meta[meta["split"]==split], split)
            for split in meta.split.unique()}
val_data = val_data['dev']


from tensorflow.keras.layers import (
    Activation,
    BatchNormalization,
    Conv1D,
 
   Dense,
    Dropout,
    Input,
    Layer,
    SpatialDropout1D,
)
from tensorflow.keras.models import Model
import tensorflow as tf

# Assuming spectral features (Batch, Time, Channels), where freq. channels are always last
TIME_AXIS = 1
STDDEV_SQRT_MIN_CLIP = 1e-10


class GlobalMeanStddevPooling1D(Layer):
    """
    Compute arithmetic mean and standard deviation of the inputs along the time steps dimension,
    then output the concatenation of the computed stats.
    """
    def call(self, inputs):
        means = tf.math.reduce_mean(inputs, axis=TIME_AXIS, keepdims=True)
        variances = tf.math.reduce_mean(tf.math.square(inputs - means), axis=TIME_AXIS)
        means = tf.squeeze(means, TIME_AXIS)
        stddevs = tf.math.sqrt(tf.clip_by_value(variances, STDDEV_SQRT_MIN_CLIP, variances.dtype.max))
        return tf.concat((means, stddevs), axis=TIME_AXIS)


def frame_layer(filters, kernel_size, strides, padding="causal", activation="relu", name="frame"):
    return Conv1D(filters, kernel_size, strides, padding=padding, activation=activation, name=name)


def segment_layer(units, activation="relu", name="segment"):
    return Dense(units, activation=activation, name=name)


from tensorflow.keras.layers import (
    Activation,
    BatchNormalization,
    Conv1D,
    Conv2D,
    Dropout,
    Dense,
    GaussianNoise,
    Input,
    Layer,
    LSTM,
    Multiply,
    Reshape,
)

from tensorflow.keras.models import Model
import tensorflow as tf


def frequency_attention(H, d_a=64, d_f=16):
    assert not H.shape[2] % d_f, "amount of frequency channels ({}) must be evenly divisible by the amount of frequency attention bins (d_f={})".format(H.shape[2], d_f)
    # Note, we assume that H.shape = (batch_size, T, d_h), but the paper assumes the timesteps come last
    x = Dense(d_a, activation="relu", use_bias=False, name="Wf_1")(H)
    F_A = Dense(d_f, activation="softmax", use_bias=False, name="Wf_2")(x)
    # Apply frequency attention on d_f bins
    F_A = Reshape((F_A.shape[1] or -1, F_A.shape[2], 1), name="expand_bin_weight_dim")(F_A)
    H_bins = Reshape((H.shape[1] or -1, d_f, H.shape[2] // d_f), name="partition_freq_bins")(H)
    H_bins = Multiply(name="freq_attention")([F_A, H_bins])
    # Merge weighted frequency bins
    H_weighted = Reshape((H.shape[1] or -1, H.shape[2]), name="merge_weighted_bins")(H_bins)
    return H_weighted



from tensorflow.keras.layers import (
    Activation,
    Dense,
    Input,
)

from tensorflow.keras.models import Model
import tensorflow as tf



def create(input_shape, num_outputs, output_activation="log_softmax", freq_attention_bins=60):
    inputs = Input(shape=input_shape, name="input")
    x = SpatialDropout1D(0.8, name="channel_dropout")(inputs)

    x = frame_layer(512, 5, 1, name="frame1")(x)
    x = frame_layer(512, 3, 2, name="frame2")(x)
    x = frame_layer(512, 3, 3, name="frame3")(x)
    x = frame_layer(512, 1, 1, name="frame4")(x)
    x = frame_layer(1500, 1, 1, name="frame5")(x)

    x = frequency_attention(x, d_f=freq_attention_bins)

    x = GlobalMeanStddevPooling1D(name="stats_pooling")(x)

    x = segment_layer(512, name="segment1")(x)
    x = segment_layer(512, name="segment2")(x)

    outputs = Dense(num_outputs, name="output", activation=None)(x)
    if output_activation:
        outputs = Activation(getattr(tf.nn, output_activation), name=str(output_activation))(outputs)
    return Model(inputs=inputs, outputs=outputs, name="x-vector-frequency-attention")



def create_model(num_freq_bins=40, num_labels=len(np.unique(meta.target))):
    m = create(
        input_shape=[None, num_freq_bins],
        num_outputs=num_labels)
    """
    m.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
            metrics=tf.keras.metrics.sparse_categorical_accuracy)
    """
    return m

with tf.device("GPU"):
    model = create_model()
    model.summary()
    pass

model.trainable = False
x = model.layers[-3].output 
x = Dense(256, activation = "relu")(x)
x = Dropout(0.35)(x)
x = Dense(128, activation = "relu")(x)
x = Dropout(0.35)(x)
predictions = Dense(4, activation = "softmax")(x)
clf1 = Model(inputs = model.input, outputs = predictions, name="clf1")

clf1.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            metrics=tf.keras.metrics.sparse_categorical_accuracy)

model.trainable = False
x = model.layers[-3].output 
x = Dense(256, activation = "relu")(x)
x = Dropout(0.35)(x)
x = Dense(128, activation = "relu")(x)
x = Dropout(0.35)(x)
predictions = Dense(4, activation = "softmax")(x)
clf2 = Model(inputs = model.input, outputs = predictions, name="clf2")


clf2.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            metrics=tf.keras.metrics.sparse_categorical_accuracy)



import numpy as np
np_rng = np.random.default_rng(1)

tf.random.set_seed(np_rng.integers(0, tf.int64.max))



import urllib.parse
from IPython.display import display, Markdown

import os

from lidbox.meta import (
    common_voice,
    generate_label2target,
    verify_integrity,
    read_audio_durations,
    random_oversampling_on_split
)

tf.random.set_seed(np_rng.integers(0, tf.int64.max))

train = pd.read_csv("train.tsv", sep="\t")
test = pd.read_csv("new_test.tsv", sep="\t")
dev = pd.read_csv("new_dev.tsv", sep="\t")


train["path"] = train["path"].apply(lambda x: x[:-3] + "mp3")
test["path"] = test["path"].apply(lambda x: x[:-3] + "mp3")
dev["path"] = dev["path"].apply(lambda x: x[:-3] + "mp3")

train["split"] = "train"
test["split"] = "test"
dev["split"] = "dev"
meta = pd.concat([train, test, dev])



meta.loc[((meta["locale"] != "kz") & ~(((meta["split"] == "dev") | (meta["split"] == "test")) & ((meta["locale"] == "ru") | (meta["locale"] == "kz") | (meta["locale"] == "en")))), "path"] = "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/" + meta.loc[((meta["locale"] != "kz") & ~(((meta["split"] == "dev") | (meta["split"] == "test")) & ((meta["locale"] == "ru") | (meta["locale"] == "kz") | (meta["locale"] == "en"))))]["locale"]  + "/clips/" + meta.loc[((meta["locale"] != "kz") & ~(((meta["split"] == "dev") | (meta["split"] == "test")) & ((meta["locale"] == "ru") | (meta["locale"] == "kz") | (meta["locale"] == "en"))))]["path"]
targets = {"kz": 0, "ru": 1, "en":2, "other":3}
meta["target"] = meta["locale"]
meta.loc[(meta["locale"] != "kz") & (meta["locale"] != "ru") & (meta["locale"]!="en"), "target"] = "other"
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5f590a130a73c.mp3"]
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5ef9bd9ba7029.mp3"]

meta["id"] = str(meta["Unnamed: 0"])
meta["target"] = meta["target"].map(targets)


meta.loc[(meta["split"] == "test") & (meta["locale"] == "ru"), "path"] = meta.loc[(meta["split"] == "test") & (meta["locale"] == "ru")]["path"].apply(lambda x: f"/tf/datasets/vox/ru_test/{x}")
meta.loc[(meta["split"] == "test") & (meta["locale"] == "ru"), "path"]
meta.loc[(meta["split"] == "test") & (meta["locale"] == "kz"), "path"] = meta.loc[(meta["split"] == "test") & (meta["locale"] == "kz")]["path"].apply(lambda x: f"/tf/datasets/vox/kz_test/{x}")
meta.loc[(meta["split"] == "test") & (meta["locale"] == "kz"), "path"] 
meta.loc[(meta["split"] == "test") & (meta["locale"] == "en"), "path"] = meta.loc[(meta["split"] == "test") & (meta["locale"] == "en")]["path"].apply(lambda x: f"/tf/datasets/vox/en_test/{x}")
meta.loc[(meta["split"] == "test") & (meta["locale"] == "en"), "path"] 


meta.loc[(meta["split"] == "dev") & (meta["locale"] == "ru"), "path"] = meta.loc[(meta["split"] == "dev") & (meta["locale"] == "ru")]["path"].apply(lambda x: f"/tf/datasets/vox/ru_dev/{x}")
meta.loc[(meta["split"] == "dev") & (meta["locale"] == "ru"), "path"]
meta.loc[(meta["split"] == "dev") & (meta["locale"] == "kz"), "path"] = meta.loc[(meta["split"] == "dev") & (meta["locale"] == "kz")]["path"].apply(lambda x: f"/tf/datasets/vox/kz_dev/{x}")
meta.loc[(meta["split"] == "dev") & (meta["locale"] == "kz"), "path"] 
meta.loc[(meta["split"] == "dev") & (meta["locale"] == "en"), "path"] = meta.loc[(meta["split"] == "dev") & (meta["locale"] == "en")]["path"].apply(lambda x: f"/tf/datasets/vox/en_dev/{x}")
meta.loc[(meta["split"] == "dev") & (meta["locale"] == "en"), "path"] 


meta.loc[meta["split"]=="test", "Unnamed: 0"] = meta.loc[meta["split"]=="test"]["path"]
meta.loc[meta["split"]=="dev", "Unnamed: 0"] = meta.loc[meta["split"]=="dev"]["path"]


meta["id"] = meta["Unnamed: 0"].apply(str)

meta.loc[meta["split"] == "test", "id"] = meta.loc[meta["split"] == "test"]["path"]


meta = meta.set_index("Unnamed: 0")
meta.loc[meta["split"]=="dev"]

meta.loc[meta["split"] == "test"] = meta.loc[(meta["split"] == "test") & (meta["target"] != 3)] 


split2ds = {split: pipeline_from_meta(meta[meta["split"]==split], split)
            for split in meta.split.unique()}


from tqdm import tqdm
EPOCHS = 100 
dev_iterator = iter(split2ds["dev"].batch(32).repeat(1000))
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4)
optimizer2 = tf.keras.optimizers.Adam(learning_rate=1e-3)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
best_acc = 0
counter = 0
MAX_PATIENCE = 5


def discrepancy(out1, out2):
    return tf.reduce_mean(tf.abs(out1 - out2))


for epoch in range(EPOCHS):
    print(f"It is epoch number: {epoch}")
    avg_loss = 0
    for i in tqdm(split2ds["train"].batch(32)):
        
        clf1.trainable = True
        clf2.trainable = True
        model.trainable = True
        model.layers[-1].trainable = False
        model.layers[-2].trainable = False
        target_data = next(dev_iterator)
        s_specs = i['logmelspec']
        t_specs = target_data['logmelspec']
        # Train for classification
        with tf.GradientTape() as tape:
            pred1 = clf1(s_specs)
            pred2 = clf2(s_specs)
            loss_classification_1 = loss_fn(i['target'], pred1)
            loss_classification_2 = loss_fn(i['target'], pred2)
            total_classification_loss = loss_classification_1 + loss_classification_2
            avg_loss += total_classification_loss
            
        train_acc_metric.update_state(i['target'], pred1)
        grads = tape.gradient(total_classification_loss, [clf1.trainable_weights, clf2.trainable_weights, model.trainable_weights])
        optimizer.apply_gradients(zip(grads[0], clf1.trainable_weights))
        optimizer.apply_gradients(zip(grads[1], clf2.trainable_weights))
        optimizer.apply_gradients(zip(grads[2], model.trainable_weights))
        # Train for discrepancy increase
        model.trainable = False
        with tf.GradientTape() as tape2:
            pred1 = clf1(s_specs)
            pred2 = clf2(s_specs)
            loss_classification_1 = loss_fn(i['target'], pred1)
            loss_classification_2 = loss_fn(i['target'], pred2)
            total_classification_loss = loss_classification_1 + loss_classification_2
            pred1 = clf1(t_specs)
            pred2 = clf2(t_specs)
            discrepancy_loss = discrepancy(pred1, pred2)
            loss = total_classification_loss - discrepancy_loss
        grads = tape2.gradient(loss, [clf1.trainable_weights, clf2.trainable_weights])
        optimizer.apply_gradients(zip(grads[0], clf1.trainable_weights))
        optimizer.apply_gradients(zip(grads[1], clf2.trainable_weights))
        # Train for discrepancy decrease
        clf1.trainable = False
        clf2.trainable = False
        model.trainable = True
        model.layers[-1].trainable = False
        model.layers[-2].trainable = False
        dloss = 0
        for k in range(3):
            with tf.GradientTape() as tape3:
                pred1 = clf1(t_specs)
                pred2 = clf2(t_specs)
                discrepancy_loss = discrepancy(pred1, pred2)
                dloss = discrepancy_loss
            grads = tape3.gradient(discrepancy_loss, model.trainable_weights)
            optimizer2.apply_gradients(zip(grads, clf1.trainable_weights))
    print(dloss)
    for batch in tqdm(val_data.batch(32)):
        val_preds = clf1(batch['logmelspec'])
        true_vals = batch['target']  
        val_acc_metric.update_state(true_vals, val_preds)
    new_acc = 0
    val_acc = val_acc_metric.result()
    new_acc += val_acc
    val_acc_metric.reset_states()
    train_acc = train_acc_metric.result()
    train_acc_metric.reset_states()
    print(f"Val acc: {val_acc}, Train acc: {train_acc}. Train loss (clf1): {avg_loss/9783}")
    for batch in tqdm(val_data.batch(32)):
        val_preds = clf2(batch['logmelspec'])
        true_vals = batch['target']  
        val_acc_metric.update_state(true_vals, val_preds)

    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    train_acc = train_acc_metric.result()
    train_acc_metric.reset_states()
    print(f"Val acc (clf2): {val_acc}.")
    new_acc += val_acc
    if new_acc / 2 > best_acc:
        print(f"Here is an improvement from {best_acc} to {new_acc / 2}\nSaving results")
        best_acc = new_acc / 2
        counter = 0
        clf1.save("clf1_3/model")
        clf2.save("clf2_3/model")
        model.save("extractor/model")
    else:
        print("No improvements")
        counter += 1
    if counter >= MAX_PATIENCE:
        break

    
        

2021-06-27 08:00:22.330 I lidbox.data.steps: Applying random resampling to signals with a random speed ratio chosen uniformly at random from [0.500, 1.500]
2021-06-27 08:00:22.454 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-27 08:00:22.466 I lidbox.data.steps: Dividing every signal in the dataset into new signals by creating signal chunks of length 3200 ms and offset 800 ms. Maximum amount of padding allowed in the last chunk is 0 ms.
2021-06-27 08:00:22.933 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-27 08:00:22.945 I lidbox.data.steps: Dividing every signal in the dataset into new signals by creating signal chunks of length 3200 ms and offset 800 ms. Maximum amount of padding allowed in the last chunk is 0 ms.
2021-06-27 08:00:23.300 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-27 08:00:23.313 I lidbox.data.steps: Dividing every signal in the dataset into new signals by

0it [00:00, ?it/s]

It is epoch number: 0


0it [00:12, ?it/s]


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: {{function_node __inference_Dataset_map_batch_extract_features_10757}} OOM when allocating tensor with shape[10176,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node PartitionedCall/stft/rfft/Pad}}]]
	 [[strided_slice_3/_138]]
  (1) Resource exhausted: {{function_node __inference_Dataset_map_batch_extract_features_10757}} OOM when allocating tensor with shape[10176,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node PartitionedCall/stft/rfft/Pad}}]]
0 successful operations.
0 derived errors ignored. [Op:IteratorGetNext]

In [25]:
!nvidia-smi

Sun Jun 27 09:02:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.116.00   Driver Version: 418.116.00   CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:59:00.0 Off |                    0 |
| N/A   41C    P0    67W / 350W |  31928MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [27]:
clf2.layers[-1].weights

[<tf.Variable 'dense_5/kernel:0' shape=(128, 4) dtype=float32, numpy=
 array([[ 0.06229074, -0.04258064, -0.11776436,  0.0842173 ],
        [ 0.13196267,  0.10288537, -0.17594309, -0.20202594],
        [ 0.15362234, -0.06542542,  0.16923791,  0.11284675],
        [ 0.10355131, -0.03872748, -0.06280154, -0.04122264],
        [-0.18054143, -0.07805274,  0.05123596,  0.16314806],
        [ 0.06057722,  0.05792367, -0.11219296, -0.12105943],
        [ 0.17948827,  0.18962532,  0.0896791 ,  0.09131058],
        [-0.02762377, -0.11449677, -0.14906867,  0.20326547],
        [ 0.14770162,  0.14940903,  0.06833924, -0.03595854],
        [ 0.2071131 ,  0.16579787, -0.14281693,  0.11614902],
        [ 0.10809997, -0.18665992,  0.15859091,  0.09405592],
        [-0.20482661, -0.05323299, -0.04854409, -0.1432973 ],
        [ 0.0270332 ,  0.1708592 ,  0.1535456 ,  0.170187  ],
        [-0.19907099, -0.09916333, -0.10690525,  0.16883467],
        [-0.17910182,  0.00298523,  0.0787999 , -0.03236267],
